In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import os
import re
import cv2

import h5py

from unidecode import unidecode 
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
# chemin des fichiers X_train.csv etc
dossier_antoine = '/Users/antoine/Documents/DataScientest/Projet/data/'
dossier_nans = ''
dossier_tristan = '../data/'
#...


# chemin des images
folder_img_antoine = '/Users/antoine/Pictures/DataScientest_Rakuten/original/image_train/'
folder_img_nans = 'C:/Users/Nans/OneDrive/Documents/Rakuten Project/images/images/image_train/'
folder_img_tristan = '../data/images/images/image_train/'
#...


dossier = dossier_tristan          # à modifier pour chacun
folder_img = folder_img_tristan   # à modifier pour chacun


In [5]:
# Loading the 2 CSVs
X = pd.read_csv(dossier + 'X_train.csv', delimiter=',', index_col=0)
y = pd.read_csv(dossier + 'Y_train.csv', delimiter=',', index_col=0)

In [6]:
# On met de côté un jeu de test auquel on ne touchera pas jusqu'à la fin, au moment de mesurer la performance du modèle retenu
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [5]:
def img_preprocessing(img_filename):

    """
    - Recadre l'image en lui retirant ses marges blanches si elle en a ;
    - Redimensionne l'image en 100 x 100 ;
    - Transforme l'image en niveaux de gris.
    """
    
    width = 100
    height = 100

    # Lecture de l'image avec OpenCV
    img = cv2.imread(folder_img + img_filename)
    
    # Conversion de l'image en niveaux de gris
    image_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Inversion des niveaux de gris
    image_inversee = cv2.bitwise_not(image_gris)

    # Recherche des contours dans l'image
    contours, hierarchy = cv2.findContours(image_inversee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        
        # Recherche du contour le plus externe
        contour_externe = max(contours, key=cv2.contourArea)

        # Recherche des coordonnées du rectangle englobant le contour externe
        x_min, y_min, w, h = cv2.boundingRect(contour_externe)

        # Rognement de l'image en utilisant les coordonnées du rectangle englobant
        cropped_image = image_gris[y_min:y_min+h, x_min:x_min+w]
        #Ou pour des images en couleurs :
        #cropped_image = img[y_min:y_min+h, x_min:x_min+w]
        #cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

        # Redimensionnement de l'image pour qu'elle ait la même taille que les autres
        cropped_image_resized = cv2.resize(cropped_image, (width, height))

        return np.array(cropped_image_resized)

    else:
        # Si aucun contour n'est trouvé, on se contente de redimensionner l'image
        image_resized = cv2.resize(image_gris, (width, height))
        #Ou pour des images en couleurs :
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #image_resized = cv2.resize(img, (width, height))
        return np.array(image_resized)

In [7]:
img_filenames = [f"image_{X.loc[idx, 'imageid']}_product_{X.loc[idx, 'productid']}.jpg" for idx in X.index]

processed_images = []

for img_filename in img_filenames:
    # Recadrage, redimensionnement et transformation en gris (cf. fonction définie plus haut)
    img = img_preprocessing(img_filename)

    # 
    processed_images.append(img)

processed_images = np.array(processed_images)

NameError: name 'img_preprocessing' is not defined

In [9]:
%%time
from joblib import Parallel, delayed

# Tristan: pretraitement des images identique avec rescale et niveau de gris mais plus rapide car avec paralellisation 

filenames = [f"image_{X.loc[idx, 'imageid']}_product_{X.loc[idx, 'productid']}.jpg" for idx in X.index]

# Définir une fonction pour traiter une image, size pour taille image et color = 'nb' pour sortie en n&b
def process_image(filename, size, color=None):
    img_scr = os.path.join(folder_img, filename)
    img = cv2.imread(img_scr)
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_inv = cv2.bitwise_not(gris)
    contours, hierarchy = cv2.findContours(img_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if color == 'nb':
        img = gris
    # si contours trouvé
    if len(contours) > 0:
        contour_exterieur = max(contours, key=cv2.contourArea)
        # Obtenir les coordonnées du rectangle englobant
        x, y, w, h = cv2.boundingRect(contour_exterieur)
        # Extraire la région d'intérêt (ROI)
        img = img[y:y+h, x:x+w]
    return cv2.resize(img, (size, size)) / 255.0

# Paralléliser le traitement des images
processed_images = Parallel(n_jobs=-1)(delayed(process_image)(filename, 100, color="nb") for filename in filenames)

CPU times: total: 41.9 s
Wall time: 1min 36s


In [10]:
X_txt = X.loc[:, ['designation', 'description']]

# Merging the column designation with the description and put everything in lowercase
X_txt["text"] = X_txt['designation'].fillna('').str.lower() + ' ' + X_txt['description'].fillna('').str.lower()

# Cleaning the dataframe by dropping the columns which are not useful anymore
X_txt.drop(['designation', 'description'], axis=1, inplace = True)

# Cleaning the text
# Deleting special character and accent with unidecode
X_txt['text'] = X_txt['text'].apply(unidecode).astype('str')
# Deleting HTML code
X_txt['text'] = X_txt['text'].str.replace(r'<[^<>]*>', '', regex=True)
# Tokenisation et deleting words with less than 3 letters
tokenizer = RegexpTokenizer(r"[a-zA-Z-]{3,}")
X_txt['text'] = X_txt['text'].apply(lambda x: tokenizer.tokenize(x.lower()))

#Deleting the stop words
stop_words = set(stopwords.words(['english','french','german']))
# Adding in addition to the stop words, the words useless for us
parasite_words_words = ['plus', 'peut', 'etre', 'tout', 'cette', 'tres']
html_code_words = ['rsquo', 'eacute', 'agrave', 'egrave', 'div', 'span', 'class', 'nbsp', 'amp', 'ecirc', 'ccedil', 'laquo', 'raquo']
stop_words.update(parasite_words_words)
stop_words.update(html_code_words)
# Function to delete stop words from our DF
def stop_words_filtering(mots) :
    tokens = []
    for mot in mots:
        if mot not in stop_words:  
            tokens.append(mot)
    return tokens
#Deleting stop words from our DF using our function
X_txt["text"] = X_txt["text"].apply(stop_words_filtering)

# Initialiser le lemmatizer
lemmatizer = WordNetLemmatizer()
# Fonction pour lemmatiser une liste de tokens
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
X_txt['text'] = X_txt['text'].apply(lemmatize_tokens)
X_txt = X_txt['text'].apply(lambda x: ' '.join(x)).astype(str).values

In [11]:
# Créer un fichier HDF5 et y enregistrer un tableau NumPy
with h5py.File('processed_data.h5', 'w') as hf:
    hf.create_dataset('img', data=processed_images, compression='gzip')
    hf.create_dataset('image_id', data=np.array(X['imageid']), compression='gzip')
    hf.create_dataset('product_id', data=np.array(X['productid']), compression='gzip')
    hf.create_dataset('label', data=np.array(y['prdtypecode']), compression='gzip')
    hf.create_dataset('text', data=X_txt, compression='gzip')